<a href="https://colab.research.google.com/github/PaulB86UK/Kaggle-Jane_Street/blob/main/Defloating_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Defloating Analysis

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Loading Data
!pip install datatable
!pip install -U keras-tuner
import datatable as dt

#Preprocessing & EDA
import pandas as pd
import numpy as np

In [6]:
#COLAB
!unzip /content/drive/MyDrive/ORDENAR/JaneStreet/train.csv.zip 
train_dt = dt.fread("/content/train.csv")
train = train_dt.to_pandas()
features = pd.read_csv("/content/drive/MyDrive/ORDENAR/JaneStreet/features.csv")
examples_test = pd.read_csv("/content/drive/MyDrive/ORDENAR/JaneStreet/example_test.csv")
example_sample_submission = pd.read_csv("/content/drive/MyDrive/ORDENAR/JaneStreet/example_sample_submission.csv")


Archive:  /content/drive/MyDrive/ORDENAR/JaneStreet/train.csv.zip
  inflating: train.csv               


**Some Insights in memory usage and faster processing**

So when pandas loads .csv files and encounters floats, it automatically makes them of data type float64.

Since the data we are given only has a precision of six digits, float64 is overkill. Using float32 would still maintain all precision but saves half the memory size!

For now using float16 for this analysis will be enough.

In [4]:

print('float\t\t bytes')
print(np.float64(-0.622475), '\t', np.float64(-0.622475).nbytes)
print(np.float32(-0.622475), '\t', np.float32(-0.622475).nbytes)
print(np.float16(-0.622475), '\t', np.float16(-0.622475).nbytes)

float		 bytes
-0.622475 	 8
-0.622475 	 4
-0.6226 	 2


In [ ]:
%%time
PATH = "/content/train.csv"
train = dt.fread(PATH).to_pandas().astype({c: np.float16 for c in train.select_dtypes(include='float64').columns})
#Result memory crash

In [5]:
def defloatizar(df,value=32):
  before = df.memory_usage().sum()
  float64_cols = df.select_dtypes(include='float64').columns
  
  if value == 32:
    mapper = {col_name: np.float32 for col_name in float64_cols}
  if value == 16:
    mapper = {col_name: np.float16 for col_name in float64_cols}
  
  df = df.astype({c: np.float16 for c in train.select_dtypes(include='float64').columns})
  after = df.memory_usage().sum()
  print(f"Memory Usage Reduction = {((before - after)/before*100).round(3)} %")  

In [6]:
%%time
train_dt = dt.fread("/content/train.csv")
train = train_dt.to_pandas()
defloatizar(train,16)

Memory Usage Reduction = 74.176 %
CPU times: user 40.8 s, sys: 10.6 s, total: 51.4 s
Wall time: 1min 10s


In [7]:
%%time
PATH = "/content/train.csv"
train = pd.read_csv(PATH, dtype=np.float16)

CPU times: user 1min 27s, sys: 4.63 s, total: 1min 31s
Wall time: 1min 39s


In [15]:
%%time
PATH = "/content/train.csv"
train = pd.read_csv(PATH, dtype=np.float16, engine= 'c')

CPU times: user 1min 29s, sys: 5.47 s, total: 1min 35s
Wall time: 1min 43s


In [16]:
def defloatizarII(PATH,datatype = np.float32):
  #check if dt is installed
  df = dt.fread(PATH)
  df = df.to_pandas()
  df = df.astype({c: datatype for c in train.select_dtypes(include='float64').columns})
  print(df.info())

In [17]:
%%time
PATH = "/content/train.csv"
defloatizarII("/content/train.csv",np.float16)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2390491 entries, 0 to 2390490
Columns: 138 entries, date to ts_id
dtypes: float64(135), int32(3)
memory usage: 2.4 GB
None
CPU times: user 39.5 s, sys: 12.7 s, total: 52.2 s
Wall time: 1min 6s
